In [ ]:
from datamodel import OrderDepth, TradingState, Order
from typing import Dict, List
import numpy as np

price_history = np.zeros(10)
positioned = False

class Trader:
    def run(self, state: TradingState) -> Dict[str, List[Order]]:
        """
        1. Given a TradingState Object in each iteration
            - Contains all trades since last iteration
            - Provides list of outstanding quotes from bots (per product)
        2. run method sends out orders to match quotes
            - If algorithm sends a buy price >= than bot quotes,
              there will be trade.
            - If algorithm sends buy order with greater qunatity than bot sell quotes
              the remaning quantity will be left as an outstanding buy quote with
              which bots will potentially continue trading with.
        3. In next iteration, TradingState will reveal whether bots traded on
           outstanding quote. If not, then quote is cancelled at end of the iteration.
        """

        """
        Reducing risk of exceeding position limits
        - Look at position limits vs current positions
        - Add/subtract instant trades (automatically matched prices)
        - Only add additional quotes outside of immediate trades to remain within limits
          as they expire anyways if no bots trade on outstanding player quote.
        """

        """
        Only method required. It takes all buy and sell orders for all symbols as an input,
        and outputs a list of orders to be sent
        """
        
        global price_history
        global positioned
        
        # Initialize the method output dict as an empty dict        
        result = {}

        # Iterate over all the keys (the available products) contained in the order depths
        for product in state.order_depths.keys():

            # Check if the current product is the 'PEARLS' product, only then run the order logic
            if product == 'BANANAS':

                current_timestamp = state.timestamp

                history_length = 10
                start_trading = 100 * history_length
                
                # Retrieve the Order Depth containing all the market BUY and SELL orders for PEARLS
                order_depth: OrderDepth = state.order_depths[product]
                
                #previous_market_trades: Trade = state.market_trades[product]
                price = 0
                count = 0
                    
                for Trade in state.market_trades[product]:
                    price += Trade.price * Trade.quantity
                    count += Trade.quantity
                    
                current_average_market_price = price / count
                price_history[:9] = price_history[1:]
                price_history[-1] = current_average_market_price
                differences = price_history[1:] - price_history[:9]
                
                discount_rate = np.logspace(0, history_length-1, num=history_length-1, base=0.7, endpoint=False)
                discount_rate = discount_rate[::-1]

                weighted_differences = differences * discount_rate
                
                expected_change = np.sum(weighted_differences) / history_length-1
                
                # Define a fair value for the product.
                acceptable_price = current_average_market_price + expected_change
                print(current_average_market_price, acceptable_price)
                acceptable_price = acceptable_price
                
                # Initialize the list of Orders to be sent as an empty list
                orders: list[Order] = []
                
                if current_timestamp >= start_trading:                
                    
                    
                    # If statement checks if there are any SELL orders in the PEARLS market
                    if len(order_depth.sell_orders) > 0:

                        # Sort all the available sell orders by their price,
                        # and select only the sell order with the lowest price
                        best_ask = min(order_depth.sell_orders.keys())
                        best_ask_volume = order_depth.sell_orders[best_ask]
                        #if positioned == True:
                        #    best_ask_volume = min(20-state.position[product], best_ask_volume)
                        #else:
                        #    best_ask_volume = min(20, best_ask_volume)

                        # Check if the lowest ask (sell order) is lower than the above defined fair value
                        if best_ask < acceptable_price and np.abs(best_ask_volume) > 0:

                            # In case the lowest ask is lower than our fair value,
                            # This presents an opportunity for us to buy cheaply
                            # The code below therefore sends a BUY order at the price level of the ask,
                            # with the same quantity
                            # We expect this order to trade with the sell order
                            print("BUY", str(-best_ask_volume) + "x", best_ask)
                            orders.append(Order(product, best_ask, -best_ask_volume))

                    # The below code block is similar to the one above,
                    # the difference is that it finds the highest bid (buy order)
                    # If the price of the order is higher than the fair value
                    # This is an opportunity to sell at a premium
                    if len(order_depth.buy_orders) != 0:
                        best_bid = max(order_depth.buy_orders.keys())
                        best_bid_volume = order_depth.buy_orders[best_bid]
                        
                        #if positioned == True:
                        #    best_bid_volume = min(np.abs(20+state.position[product]), best_ask_volume)
                        #else:
                        #    best_ask_volume = min(20, best_bid_volume)
                        
                        if best_bid > acceptable_price and best_bid_volume > 0:
                            print("SELL", str(best_bid_volume) + "x", best_bid)
                            orders.append(Order(product, best_bid, -best_bid_volume))

                    # Add all the above orders to the result dict
                    result[product] = orders
                    if result[product] != []:
                        positioned = True

                # Return the dict of orders
                # These possibly contain buy or sell orders for PEARLS
                # Depending on the logic above
        return result
        